<a href="https://colab.research.google.com/github/jegazhu/python-projects/blob/main/GeoData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open(r'/content/where.data')
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")


Retrieving http://py4e-data.dr-chuck.net/json?address=Harvard+University&key=42
Retrieved 1468 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Massachusetts+Institute+of+Technology&key=42
Retrieved 2142 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Stanford+University&key=42
Retrieved 1918 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Cambridge&key=42
Retrieved 52 characters {    "results" : [],

Retrieving http://py4e-data.dr-chuck.net/json?address=California+Institute+of+Technology&key=42
Retrieved 2148 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Princeton+University&key=42
Retrieved 1808 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Oxford&key=42
Retrieved 1783 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Yale+U

In [3]:
import sqlite3
import json
import codecs

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")

Cambridge, MA, USA 42.3770029 -71.11666009999999
77 Massachusetts Ave, Cambridge, MA 02139, USA 42.360091 -71.09416
450 Serra Mall, Stanford, CA 94305, USA 37.4274745 -122.169719
1200 E California Blvd, Pasadena, CA 91125, USA 34.1376576 -118.125269
Princeton, NJ 08544, USA 40.3430942 -74.65507389999999
Oxford OX1 2JD, UK 51.7548164 -1.2543668
New Haven, CT 06520, USA 41.3163244 -72.92234309999999
New York, NY 10027, USA 40.8075355 -73.9625727
Berkeley, CA, USA 37.8718992 -122.2585399
5801 S Ellis Ave, Chicago, IL 60637, USA 41.7886079 -87.5987133
Ithaca, NY 14850, USA 42.4534492 -76.4735027
Philadelphia, PA 19104, USA 39.9522188 -75.1932137
7-chōme-3-1 Hongō, Bunkyo City, Tokyo 113-8654, Japan 35.7126775 139.761989
Baltimore, MD 21218, USA 39.3299013 -76.6205177
Rämistrasse 101, 8092 Zürich, Switzerland 47.37638889999999 8.547628
Yoshidahonmachi, Sakyo Ward, Kyoto, 606-8501, Japan 35.0262444 135.7808218
Herzl St 234, Rehovot, Israel 31.90375409999999 34.8080315
Los Angeles, CA 90095, 